<a href="https://colab.research.google.com/github/DhayalanCS/TensorFlow/blob/main/textgenerationshakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
#decoding the text from the file
text = open(path_to_file, 'rb').read().decode(encoding ='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
print(text[:300]) #first300characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [ ]:
#unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [ ]:
# here we are processing the text by vectorizing it. The words are given a numerical id.
example_text = ['abcdefg','xyz']
chars = tf.strings.unicode_split(example_text, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
#creating the preprocessing layer
#tokens to character ids
ids_from_chars = preprocessing.StringLookup(
    vocabulary = list(vocab)
)

ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary = ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis= -1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)


In [ ]:
#the prediction task
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([20, 49, 58, ..., 47, 10,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
#tensor slices function to convert the text vector into a stream of character indices.

for ids in ids_dataset.take(15):
  print(chars_from_ids(ids).numpy().decode('utf-8'))


F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [ ]:
#batch method lets you easily convert these individual characters to sequences of the desired size.

seq_length = 100
example_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())
  

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
#creating training examples and targets
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 5024

#This model has three layers:

#tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
#tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
#tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)




In [ ]:
#checking the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 67) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()    


Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17152     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices

array([40, 44, 48, 39, 47,  2, 51, 22, 49, 46, 17, 28, 13, 53, 21, 57, 51,
       31, 36,  2, 58, 53, 42, 37, 23, 25, 28, 14, 19, 10, 19, 53, 19, 22,
       57, 31, 14, 16, 29, 63, 37, 27, 37, 35,  2, 22, 40, 53, 25,  2, 12,
        8, 63,  7, 13, 25, 15, 33, 27, 34, 62, 55, 63, 20, 15,  9, 64, 19,
       27, 61, 42,  1, 39,  0, 55, 33, 22, 27, 30, 38, 42, 61, 64, 39, 26,
       18, 52,  8, 53, 27, 19, 35, 26, 14, 22,  9, 38, 29, 47,  6])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"our dies,\nOr my shamed life in his dishonour lies:\nThou kill'st me in his life; giving him breath,\nT"

Next Char Predictions:
 b"ZdhYg\nkHifCN;mGqkQV\nrmbWIKN?E.EmEHqQ?BOwWMWU\nHZmK\n:,w';KASMTvowFA-xEMub[UNK]YoSHMPXbuxYLDl,mMEUL?H-XOg&"


Train the model
At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the from_logits flag.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.2048364


In [ ]:
tf.exp(mean_loss).numpy()

67.00963

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 894s 5s/step - loss: 3.2955
Epoch 2/20
172/172 [==============================] - 895s 5s/step - loss: 2.0778
Epoch 3/20
172/172 [==============================] - 895s 5s/step - loss: 1.7565
Epoch 4/20
172/172 [==============================] - 888s 5s/step - loss: 1.5726
Epoch 5/20
172/172 [==============================] - 890s 5s/step - loss: 1.4573
Epoch 6/20
172/172 [==============================] - 883s 5s/step - loss: 1.3844
Epoch 7/20
172/172 [==============================] - 883s 5s/step - loss: 1.3287
Epoch 8/20
172/172 [==============================] - 883s 5s/step - loss: 1.2779
Epoch 9/20
172/172 [==============================] - 878s 5s/step - loss: 1.2361
Epoch 10/20
172/172 [==============================] - 886s 5s/step - loss: 1.1933
Epoch 11/20
172/172 [==============================] - 873s 5s/step - loss: 1.1481
Epoch 12/20
172/172 [==============================] - 853s 5s/step - loss: 1.1066
Epoch 13/20
1

In [ ]:
#Generate Text

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
'Tis for my liege, what he would fetch her sister
save the Four Rosaline, whom then, of wash your body
to such as often and the heavy back:
'Tis more than you can harment me;
And if Letis another day hath power to this?

BAPTISTA:
Good night, dead call, at least,
And the nobicilier of his useal music at behavior:
And so come to her recompension on Fortun,
Where being patterly as a prisoner,
O, but what is not Rutland more fierced and great love?

BIANCA:
Now justice men, my lord, let me an end;
Some say he has in possess.

First Conspirator:
So, mighty soul.

TYRREL:
My lord, your souls not build to die.

CAMILLO:

BIANCA:
If we will be, love them most aumed
To bloody things were from that have before thee; make up my brother:
Nor shrinks it open, let me lay in bribre.
Best fortain our mildness in the bind this dreams,
That he consents in mine arms. I have been down thee:
That's the way into your ablector; selute haste,
And sligh their smiles on the earth's company
As to your ro

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nStir, there is such as wheels sit with their frailing bed.\nHer sux his poiton: thou hast slept: then, I would kiss\nAt frame you something new.\nO manage Kents, the temple near 'we?\nOne world! she was the devil's commandered\nthat he hath lost either of your souls\nTo strew him up that hungman moved his head.\n\nDUKE OF YORK:\nIt shall be profess, but in a serpen of all nister gates:\nNow sways it with me an apprehension.\nGood nakes, you the devil's death,\nWhich then comes on my brother is come along.\n\nDUKE OF YORK:\nIt were none.\n\nPETRUCHIO:\nCome but thee; there is my life may nine.\n\nPAULINA:\nEre sent you supory.\n\nKING HENRY VI:\nFor what, lies, well we here: God keep her use we heard\nMore than a pipour of his love.\n\nHASTINGS:\nI think, I hear, sir, by the regal spirit,\n'Be seem to bed to murder: mine is a traitor of the world;\nAnd yet shall sting me that e'er made thee back.\n\nPAULINA:\nSir, here some shall be well.\n\nBAPTISTA:\nAy, good my 

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')#exporting the model

states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


ROMEO:
What fates well,
'I was adventure.

POMPEY:
I son to him, and he shall quit like liberty.

GLOUCEST


In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 885s 5s/step - loss: 2.7245


In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2131
Epoch 1 Batch 50 Loss 2.0703
Epoch 1 Batch 100 Loss 1.9890
Epoch 1 Batch 150 Loss 1.8595

Epoch 1 Loss: 1.9961
Time taken for 1 epoch 884.78 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8008
Epoch 2 Batch 50 Loss 1.7652
Epoch 2 Batch 100 Loss 1.7051
Epoch 2 Batch 150 Loss 1.6534

Epoch 2 Loss: 1.7176
Time taken for 1 epoch 879.39 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6274
Epoch 3 Batch 50 Loss 1.5706
Epoch 3 Batch 100 Loss 1.5575
Epoch 3 Batch 150 Loss 1.5669

Epoch 3 Loss: 1.5560
Time taken for 1 epoch 875.80 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5081
Epoch 4 Batch 50 Loss 1.5033
Epoch 4 Batch 100 Loss 1.4758
Epoch 4 Batch 150 Loss 1.4960

Epoch 4 Loss: 1.4571
Time taken for 1 epoch 881.18 sec
_________________________________________________________________